In [7]:
import gzip, ujson, random
import toolz
import cPickle as c
import nltk.tokenize
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
import dill

In [8]:
import gzip,ujson
filepath_cat = 'yelp_train_academic_dataset_business.json.gz'
businesscat = list()
with gzip.open(filepath_cat,'rb') as f:
    for line in f:
        temp = ujson.loads(line)
        if 'Restaurants' in temp['categories']:
            businesscat.append(temp['business_id'])
print len(businesscat)

12876


In [59]:
def getjsontext(d):
    return [toolz.keyfilter(lambda k: k == "text", d)]

def load_json_yelp(filepath):
    data=list()
    with gzip.open(filepath, 'rb') as f:
        for line in f:
            temp = ujson.loads(line)
            if temp['business_id'] in businesscat:
                #data['text'].append(toolz.keyfilter(lambda k: k == "text", line))
                data.append(temp['text'])
                #data['stars'].append(temp['stars'])
    return data

def tokenizereview(str):
    return nltk.tokenize.sent_tokenize(str)

#data loading
filepath = 'yelp_train_academic_dataset_review.json.gz'
yelp_review = load_json_yelp(filepath)

In [10]:
print len(yelp_review)
#print len(yelp_data['text'])
#print len(yelp_data['stars'])

574278
574278


In [12]:
with open('yelp_restaurants_review.data','wb') as output:
    dill.dump(yelp_data,output)

In [26]:
test_sample = yelp_data['text'][:10]

In [57]:
#print test_sample

In [125]:
import numpy as np
test_vec2 = CountVectorizer(ngram_range=(2,2))
b=test_vec2.fit_transform(test_sample).sum(axis=0)
print type(b)
b=b+90
ts = float(b.sum())
#pw=[float(i)/float(ts) for i in b]
pw2 = b/ts
print b.shape
#test_vec.get_feature_names()
test_vec = CountVectorizer(ngram_range=(1,1))
a=test_vec.fit_transform(test_sample).sum(axis=0)
ts = float(a.sum())
pw=a/ts
#pw2=[float(i)/float(ts) for i in a]
print a.shape
fn1=test_vec.get_feature_names()
fn2=test_vec2.get_feature_names()
ng2list = list()
for i in fn2:
    sp = i.split()
    if sp[0] in fn1:
        pw1ind=fn1.index(sp[0])
    if sp[1] in fn1:
        pw2ind=fn1.index(sp[1])
    pw1w2ind = fn2.index(i)
    bigram = (i,pw2[0,pw1w2ind]/pw[0,pw1ind]/pw[0,pw2ind])
    ng2list.append(bigram)
print ng2list

<class 'numpy.matrixlib.defmatrix.matrix'>
(1, 782)
(1, 394)
[(u'00 father', 1007.0097953913946), (u'10 high', 1007.0097953913946), (u'12 added', 1007.0097953913946), (u'50s take', 251.75244884784865), (u'51 be', 201.40195907827894), (u'94 just', 251.75244884784865), (u'about 10', 251.75244884784865), (u'about dinner', 62.938112211962164), (u'about eating', 125.87622442392433), (u'about year', 125.87622442392433), (u'across the', 16.508357301498272), (u'add alot', 1007.0097953913946), (u'added breakfast', 335.66993179713148), (u'addition of', 53.000515546915501), (u'advice leave', 251.75244884784865), (u'ago and', 17.982317774846333), (u'ago but', 125.87622442392433), (u'agreement then', 503.50489769569731), (u'ahead and', 35.964635549692666), (u'all the', 5.5632559404316515), (u'all you', 41.958741474641442), (u'allowed smoking', 1007.0097953913946), (u'alone is', 91.546345035581325), (u'alot of', 53.000515546915501), (u'also kind', 1007.0097953913946), (u'always been', 503.5048976956

In [126]:
print 'start counting'
vec_ng1 = CountVectorizer(ngram_range=(1,1))
vec_ng2 = CountVectorizer(ngram_range=(2,2))
print 'start calculate freqency'
count_1 = vec_ng1.fit_transform(yelp_review)
print 'start sum'
count_sum = count_1.sum(axis=0)
print 'start todense'
count_1 = count_sum
print 'start smoothing'
count_1 = count_1+90
freq_1 = count_1/float(count_1.sum())
print 'start counting ngram2'
count_2 = vec_ng2.fit_transform(yelp_review)
count_2 = count_2.sum(axis=0)
freq_2 = count_2/float(count_2.sum())
print 'getting feature names'
index1 = vec_ng1.get_feature_names()
index2 = vec_ng2.get_feature_names()
bigramlist = list()
count=0
print 'start calculateing bigram factor'
for i in index2:
    count+=1
    sp = i.split()
    if sp[0] in index1:
        pw1ind=index1.index(sp[0])
    if sp[1] in index1:
        pw2ind=index1.index(sp[1])
    pw1w2ind = index2.index(i)
    bigram = (i,pw2[0,pw1w2ind]/pw[0,pw1ind]/pw[0,pw2ind])
    bigramlist.append(bigram)
    if count%1000==0:
        print count
sort_bigram = sorted(bigramlist,key = lambda (a,b):-b)[:100]

start counting
start calculate freqency
start sum
start todense
start smoothing
start counting ngram2


MemoryError: 